In [1]:
# Import monai and ITK
import monai
import itk
import numpy as np


In [2]:
from monai.apps import DecathlonDataset

# Set the directory where you want to download the dataset
root_dir = "./data"

# Download the dataset
DecathlonDataset(root_dir=root_dir, task="Task08_HepaticVessel", section="training", download=True)


2023-12-21 00:08:40,874 - INFO - Verified 'Task08_HepaticVessel.tar', md5: 641d79e80ec66453921d997fbf12a29c.
2023-12-21 00:08:40,875 - INFO - File exists: data/Task08_HepaticVessel.tar, skipped downloading.
2023-12-21 00:08:40,876 - INFO - Non-empty folder exists in data/Task08_HepaticVessel, skipped extracting.


monai.transforms.io.dictionary LoadImaged.__init__:image_only: Current default value of argument `image_only=False` has been deprecated since version 1.1. It will be changed to `image_only=True` in version 1.3.
Loading dataset:  31%|███       | 75/243 [01:08<05:49,  2.08s/it]

: 

In [ ]:
import itk
import os

# Directory paths for images and masks
data_dir = "./data/Task08_HepaticVessel/"
images_dir = os.path.join(data_dir, "imagesTr")
masks_dir = os.path.join(data_dir, "masksTr")

# Function to compute statistics for a list of images
def compute_statistics(image_files):
    all_means = []
    all_stds = []
    all_mins = []
    all_maxs = []

    for img_file in image_files:
        image = itk.imread(img_file)
        img_arr = itk.array_from_image(image)

        # Compute statistics for each image
        img_mean = img_arr.mean()
        img_std = img_arr.std()
        img_min = img_arr.min()
        img_max = img_arr.max()

        all_means.append(img_mean)
        all_stds.append(img_std)
        all_mins.append(img_min)
        all_maxs.append(img_max)

    return all_means, all_stds, all_mins, all_maxs

# Get file paths for images and masks
image_files = [os.path.join(images_dir, file) for file in os.listdir(images_dir)]
mask_files = [os.path.join(masks_dir, file) for file in os.listdir(masks_dir)]

# Compute statistics for images
image_means, image_stds, image_mins, image_maxs = compute_statistics(image_files)
mask_means, mask_stds, mask_mins, mask_maxs = compute_statistics(mask_files)

# Print statistics
print("Image Statistics:")
print(f"Mean: {sum(image_means) / len(image_means):.2f}")
print(f"Standard Deviation: {sum(image_stds) / len(image_stds):.2f}")
print(f"Min Value: {min(image_mins)}")
print(f"Max Value: {max(image_maxs)}")

print("\nMask Statistics:")
print(f"Mean: {sum(mask_means) / len(mask_means):.2f}")
print(f"Standard Deviation: {sum(mask_stds) / len(mask_stds):.2f}")
print(f"Min Value: {min(mask_mins)}")
print(f"Max Value: {max(mask_maxs)}")


Fast marching level-set segmentation


In [ ]:
import itk
import matplotlib.pyplot as plt

def fast_marching_segmentation(input_filepath,
                               seed_x=56, seed_y=92,
                               basin_minimum=1.55,
                               border_minimum=2.45,
                               stopping_value=100,
                               time_threshold=100,
                               output_filepath='output_segmentation.png'):

    input_image = itk.imread(input_filepath, pixel_type=itk.F)

    smoothing_filter = itk.CurvatureAnisotropicDiffusionImageFilter.New(input_image)
    smoothing_filter.SetTimeStep(0.125)
    smoothing_filter.SetNumberOfIterations(5)
    smoothing_filter.SetConductanceParameter(3.0)

    gradient_filter = itk.GradientMagnitudeRecursiveGaussianImageFilter.New(smoothing_filter)
    gradient_filter.SetSigma(1.0)

    sigmoid_filter = itk.SigmoidImageFilter.New(gradient_filter)
    sigmoid_filter.SetOutputMinimum(0.0)
    sigmoid_filter.SetOutputMaximum(1.0)
    alpha = -(float(border_minimum) - float(basin_minimum)) / 3.0
    beta = (float(border_minimum) + float(basin_minimum)) / 2.0
    sigmoid_filter.SetAlpha(alpha)
    sigmoid_filter.SetBeta(beta)
    fastmarching = itk.FastMarchingImageFilter.New(sigmoid_filter)

    dimension = input_image.GetImageDimension()
    IndexType = itk.Index[dimension]
    seed_position = IndexType()
    seed_position[0] = int(seed_x)
    seed_position[1] = int(seed_y)

    NodeType = itk.LevelSetNode[itk.F, dimension]
    node = NodeType()
    node.SetValue(0.)
    node.SetIndex(seed_position)
    NodeContainer = itk.VectorContainer[itk.UI, NodeType]
    seeds = NodeContainer.New()
    seeds.Initialize()
    seeds.InsertElement(0, node)
    fastmarching.SetTrialPoints(seeds)

    fastmarching.SetOutputSize(input_image.GetBufferedRegion().GetSize())
    fastmarching.SetStoppingValue(float(stopping_value))

    output_type = itk.Image[itk.UC, dimension]
    thresholder = itk.BinaryThresholdImageFilter[input_image, output_type].New(fastmarching)
    thresholder.SetLowerThreshold(0.0)
    thresholder.SetUpperThreshold(float(time_threshold))
    thresholder.SetOutsideValue(0)
    thresholder.SetInsideValue(255)

    # Update the filter
    thresholder.Update()

    output_image = thresholder.GetOutput()

    # Save the segmented output
    itk.imwrite(output_image, output_filepath)

    # Plot the segmented output
    plt.imshow(itk.GetArrayViewFromImage(output_image), cmap='gray')
    plt.axis('off')
    plt.title('Segmentation Result')
    plt.show()
